In [1]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(limma))
suppressPackageStartupMessages(library(beeswarm))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(pheatmap))

In [2]:
homedir = "/nfs/lab/projects/pbmc_snATAC/analysis_v2"

In [3]:
setwd(homedir)

In [4]:
ids = c(2:5, 8:10,12:14)

#### Normalize the count table using the Deseq2 functions
Note  I am using the full matrix here- not fileterd for minimim coverage and the transform - in rasqual first filtered the matrix and then calculate the size-factors - I should not matter in this case as it is just for visualization purposes- 

In [230]:
reso = "broad"
maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]


In [59]:
reso = "fine"
maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]


In [60]:
celltypes

[1] "act_cd4_t"   "adaptive_NK" "cDC"         "cMono"       "cyto_cd8_t" 
 [6] "cyto_nk"     "iMono"       "mem_b"       "mem_cd8_t"   "mkc"        
[11] "naive_b"     "naive_cd4_t" "naive_cd8_t" "ncMono"      "tReg"

In [232]:
for (c in celltypes [6]){

mat = read.table(paste0(maindir,"/", c, "/", c, ".count_matrix"), header=T)
colnames(mat)[7:ncol(mat)] = paste0("PBMC", ids)
rownames(mat) = mat[,1]
mat           = mat[,7:ncol(mat)]
#mat_vst = vst(as.matrix(mat), blind=FALSE)
mat_rlog = rlog(as.matrix(mat), blind=FALSE)

rownames(mat_rlog) = rownames(mat)

write.table(mat_rlog, paste0('example_boxplots/', c, ".rlog.count_matrix"), sep="\t", quote=F, row.names=T)
}

In [188]:
reso = "fine"
maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]
for (c in celltypes ){

mat = read.table(paste0(maindir,"/", c, "/", c, ".count_matrix"), header=T)
colnames(mat)[7:ncol(mat)] = paste0("PBMC", ids)
rownames(mat) = mat[,1]
mat           = mat[,7:ncol(mat)]
#mat_vst = vst(as.matrix(mat), blind=FALSE)
mat_rlog = rlog(as.matrix(mat), blind=FALSE)

rownames(mat_rlog) = rownames(mat)

write.table(mat_rlog, paste0('example_boxplots/', c, ".rlog.count_matrix"), sep="\t", quote=F, row.names=T)
}

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

-- note: fitType='parametric', but the d

In [215]:
reso = "bulk"
maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]
for (c in celltypes ){

mat = read.table(paste0(maindir,"/", c, "/", c, ".count_matrix"), header=T)
colnames(mat)[7:ncol(mat)] = paste0("PBMC", ids)
rownames(mat) = mat[,1]
mat           = mat[,7:ncol(mat)]
#mat_vst = vst(as.matrix(mat), blind=FALSE)
mat_rlog = rlog(as.matrix(mat), blind=FALSE)

rownames(mat_rlog) = rownames(mat)

write.table(mat_rlog, paste0('example_boxplots/', c, ".rlog.count_matrix"), sep="\t", quote=F, row.names=T)
}

#### List of all tested SNPs and genotypes

In [238]:
celltypes

character(0)

In [239]:
for (reso in c( "broad", "fine", "bulk")){

maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]
    
for (cell in celltypes) {
cat (cell, "\n")
    }
    }

b 
cd4_t 
cd8_t 
mono 
nk 
t 
act_cd4_t 
adaptive_NK 
cDC 
cMono 
cyto_cd8_t 
cyto_nk 
iMono 
mem_b 
mem_cd8_t 
mkc 
naive_b 
naive_cd4_t 
naive_cd8_t 
ncMono 
tReg 
bulk 


In [234]:
snps = data.frame()

for (reso in c( "broad", "fine", "bulk")){

maindir     = paste0('rasqual_eur/', reso)
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]
    
for (cell in celltypes) {

setwd(paste(homedir, maindir, cell, "vcfs_peaksonly", sep="/"))

 system(paste0("bcftools query -f '%CHROM\\t%POS\\t%ID\\t%REF\\t%ALT[\\t%GT]\\n' ", 
                "full.ase.filtered.vcf.gz > full.ase.filtered.genotypes"))

snps_cell = read.table("full.ase.filtered.genotypes", header=F, stringsAsFactors = F)
snps = rbind(snps, snps_cell)
    }
    }

setwd(homedir)
snps = snps[!duplicated(snps),]

snps <- replace(snps, snps == "/", "|")
snps <- replace(snps, snps == "1|1", 2)
snps <- replace(snps, snps == "0|1", 1)
snps <- replace(snps, snps == "1|0", 1)
snps <- replace(snps, snps == "0|0", 0)

samples = paste0("PBMC",  ids)
colnames(snps) = c("CHR", "POS", "ID", 'REF', 'ALT', samples) 

genotypes           = as.matrix(snps[,samples])
rownames(genotypes) = snps$ID
dim(genotypes)

[1] 191116     10

In [236]:
write.table(genotypes, "example_boxplots/All_tested_genotypes.tsv", sep="\t", quote=F)

In [237]:
write.table(genotypes, "anova_interaction/All_tested_genotypes.tsv", sep="\t", quote=F)

### Boxplot function

In [5]:
clrs = read.table("peaks/color_scheme.tsv")

In [6]:
clrs = rbind(clrs, matrix( c('orange2', 'b','tan4','nk','#7FC97F','mono', 'magenta4', "t"), ncol=2, byrow=T))

In [49]:
plot_boxplot = function(peak , snp, cell , zscore=TRUE,suffix="",
                        gttab = "example_boxplots/All_tested_genotypes.tsv",
                        smr_dir='summarized_caqtls', 
                        tbl_dir = 'example_boxplots', YLIM=NULL ){
    
   sumtab = paste0(smr_dir,"/",cell ,'_caqtl_sumstats',suffix , '.tsv')
   counts = paste0(tbl_dir,'/',cell ,'.rlog.count_matrix')
    col   = as.character(clrs[,1] [clrs[,2]== cell])
    
             
pip = 'example_boxplots/tmp'              
system(paste("grep", peak, sumtab, ">", pip))
                 
if(file.size(pip)>0) {  
                 rasq = read.table(pip)
                  rasq = subset(rasq, grepl(snp,rasq$V2))
    
if(nrow(rasq)>0) {
                 q  = signif(10^(rasq[,"V10"]),2)
                 rs = rasq$V30  

genot = c(paste0(rasq[,5],rasq[,5]), paste0(rasq[,5],rasq[,6]), paste0(rasq[,6],rasq[,6]))
names(genot) = c(0,1,2)


} else {   
    q  = "NA"
    rs = "snpnot tested"
    genot = c(0,1,2)
    names(genot) = c(0,1,2)
} 
    
     
    df = t(rbind(read.table(pipe(paste("grep", peak, counts)))[-1],
                 read.table(pipe(paste("grep", snp, gttab)))[-1]))

    ngt= genot[names(genot) %in% unique(df[,2])]

if(zscore == TRUE ){ df[,1] = scale(df[,1]) }
    
colnames(df) = c('peak_counts','genotype')
boxplot(peak_counts~ genotype ,df, las=1, col = col,
        outline=F, xlab=NA, xaxt="n" ,yaxt="n", ylim=YLIM)
axis(2,tck=-0.1, las=1, cex.axis=0.8, lwd=0.6)
axis(1,tick=F, las=2, cex.axis=0.8, lwd=0.6,
labels = ngt,  at=1:length(ngt))
beeswarm(peak_counts~ genotype ,df, add=T, pch=21, bg="white", cex=0.5)
mtext(paste0("q=",q ), cex=0.5)
mtext(peak, line=0.5, cex=0.4)  
mtext(rs, line=2, cex=0.5, side=1)         

    
   } else{
      
    plot.new()
  }
    
            
               
}

In [ ]:
#awk -v OFS="\t" '{print "chr"$1,$2-1,$2}' snp_list > snp_list.bed
#bedtools intersect -a snp_list.bed -b ../rasqual/bulk/bulk/bulk.filtered.bed -wa -wb > snp_list.peaks.bed 

### Make figures

In [31]:
examples = read.table('example_boxplots/snp_list.peaks.bed')

examples$snps  = paste0(examples$V1, ":" , examples$V3,":" )
examples$peaks = paste0(examples$V4, ":" , examples$V5 -1 ,"-" ,examples$V6) 

examples$CAT = c('private','private','private','private','divergent', 'divergent', 'coacc', 
'gwas', 'gwas', 'gwas','gwas','gwas','private','private','private','private','divergent')
examples$important = c(F,F,F,F,F,T,F,T,T,T,T,T,F,F,F,F,F)

In [251]:
imp = subset(examples, important==T)

In [262]:
reso        = "broad"
celltypes   = c("bulk", 'b', 'mono', 't', 'nk')

In [270]:
pdf("figures/Boxplot_divergent.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for(x in 1:nrow(imp)){
    snp  = imp$snps[x]
    peak = imp$peaks[x]
#plot.new()
for(c in  celltypes){
    
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=c(-2,1.8))
    
    }
    plot.new()
    plot.new()
    
    }

dev.off()

png 
  2

In [305]:
anova_results = read.csv("anova_interaction/results_anova_broad.csv", row.names=1)

In [277]:
priv = c("rs13294415","rs10888395","rs10511304", "rs1475159" )

In [279]:
private = subset(anova_results, rsID %in% priv)

In [286]:
ylims =list(c(-2.1, 1.5), c(-1.5, 1.5), c(-1.8, 1.6), c(-1.5,2.2))

In [287]:
pdf("figures/Boxplot_private.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for(x in 1:nrow(private)){
    snp  = private$varID[x]
    peak = private$Feature[x]
    cell_sel = as.character(private$cell[x])
#plot.new()
for(c in  c("bulk", cell_sel)){
    
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=ylims[[x]])
    
    }
    plot.new()
    plot.new()
    
    }

dev.off()

png 
  2

In [310]:
tc = subset(anova_results, private ==FALSE & exclusive==TRUE & aov_qval<0.1 & n_tested_cells==4 )

In [316]:
pdf("figures/Boxplot_new.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for(x in 8){
    snp  = tc$varID[x]
    peak = tc$Feature[x]
   
#plot.new()
for(c in  celltypes){
    
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=c(-2.2, 1.7))
    
    }
 
    }

dev.off()

png 
  2

### coaccessible example

In [8]:
reso        = "broad"
celltypes   = c("bulk", 'b', 'mono', 't', 'nk')

In [27]:
pdf("figures/Boxplot_coacc.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
    c = "t"
    snp  = 'chr14:95412171'
    peak = 'chr14:95235918-95236970'
   
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=c(-2.2, 1.7))
    
    peak = 'chr14:95411884-95413043'
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=c(-2.2, 1.7))
    dev.off()

png 
  2

### gwas examples

In [37]:
gwas = subset(examples, CAT=="gwas")

In [38]:
gwas = gwas [-5,c('snps','peaks')]

In [40]:
gwas = rbind(gwas, c('chr5:55444683:','chr5:55254433-55255382' ))

In [43]:
cellule = list(  c("bulk", "t", "naive_cd4_t", "naive_cd8_t"), 
                 c("bulk", 't', 'cyto_cd8_t','b','mono','act_cd4_t','mem_b'),
                 c("bulk",'cMono','mem_cd8_t','naive_cd8_t','cDC','act_cd4_t','ncMono','naive_cd4_t','tReg','cyto_cd8_t'),
                 c("bulk", "t", "act_cd4_t"),  c("bulk", "t", "act_cd4_t"))

In [46]:
pdf("figures/Boxplot_gwas_vst.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)

for (i in 1:5){
cells_sel = cellule[[i]]
    snp  = gwas$snps[i]
    peak = gwas$peaks[i]
    for(c in cells_sel){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = FALSE)
 }
}
dev.off()

png 
  2

In [50]:
pdf("figures/Boxplot_gwas_test.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)

for (i in 3){
cells_sel = c("bulk", "t", "mono", "b", 'nk')
    snp  = gwas$snps[i]
    peak = gwas$peaks[i]
    for(c in cells_sel){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
 }
}
dev.off()

png 
  2

In [56]:
pdf("figures/Boxplot_gwas_new.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)


cells_sel = c("bulk", "mono", "t", "b", 'nk','mem_cd8_t')
    snp  = 'chr7:28724374:'
    peak = 'chr7:28724213-28725891'
    for(c in cells_sel){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
}
    snp  = 'chr12:56473808:'
    peak = 'chr12:56472735-56474339'
    for(c in cells_sel){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
}

    snp  = 'chr2:228670476:'
    peak = 'chr2:228669766-228670866'
    for(c in cells_sel){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
}
dev.off()

png 
  2

In [61]:
cells_sel = c("bulk", "mono", "t", "b", 'nk')

In [62]:
pdf("figures/Boxplot_cc20.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)

snp  = 'chr2:228670476:'
    peak = 'chr2:228669766-228670866'
    for(c in c(cells_sel,celltypes)){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
}
dev.off()

png 
  2

In [73]:
pdf("figures/Boxplot_supp_caqtls.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for (i in 13:17){
snp  = examples$snps[i]
peak = examples$peaks[i]
    for(c in c(cells_sel,celltypes)){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=NULL, zscore = TRUE)
}
    }
dev.off()

png 
  2

In [83]:
ex2 = data.frame(peaks = c('chr12:47643727-47644530', 'chr4:108785014-108785669',
                        'chr6:1286253-1286584','chr11:9594342-9595642', 
                           'chr1:203642938-203643753','chr17:41400227-41401288','chr6:36354187-36356090'),
                 snps = c('chr12:47644392:', 'chr4:108785332:','chr6:1286450:',
                          'chr11:9594690:','chr1:203643268:','chr17:41401202:','chr6:36355610:'))


In [105]:
pdf("figures/Boxplot_supp_caqtls3.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for (i in 7){
snp  = ex2$snps[i]
peak = ex2$peaks[i]
    for(c in c(cells_sel,celltypes)){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=c(-2.2,2.2), zscore = TRUE)
}
    }
dev.off()

png 
  2

In [91]:
 ex1 = as.data.frame(matrix(c('chr5:157506340:','chr5:157506084-157506508',
                'chr12:47644392:','chr12:47643727-47644530',
                'chr6:1286450:', 'chr6:1286253-1286584',
                'chr9:108966249:','chr9:108966003-108966777'), ncol=2, byrow=T))
ex1$cell = c('naive_b', 'naive_cd4_t', 'cMono','cyto_nk')

In [103]:
ylims = list(c(-2.2,1.5),c(-1.8,1.8),c(-1,2.2),c(-1.6,1.6))

In [104]:
pdf("figures/Boxplot_supp_fine.pdf")
par(mfrow=c(6,8), mar = c(2,2,2,2), lwd=0.6)
for (i in 1:4){
snp  = ex1$V1[i]
peak = ex1$V2[i]
    for(c in c("bulk",ex1$cell[i])){
    plot_boxplot (peak , snp, c,suffix="_run3",  smr_dir='summarized_caqtls/run3', YLIM=ylims[[i]], zscore = TRUE)
}
    }
dev.off()

png 
  2

In [111]:
examples

V1,V2,V3,V4,V5,V6,snps,peaks,CAT,important
<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>
chr9,77669151,77669152,chr9,77668569,77669180,chr9:77669152:,chr9:77668568-77669180,private,FALSE
chr6,164141648,164141649,chr6,164141435,164141902,chr6:164141649:,chr6:164141434-164141902,private,FALSE
chr1,150762170,150762171,chr1,150761825,150762301,chr1:150762171:,chr1:150761824-150762301,private,FALSE
chr8,2143417,2143418,chr8,2142843,2143474,chr8:2143418:,chr8:2142842-2143474,private,FALSE
chr12,113671188,113671189,chr12,113670656,113671680,chr12:113671189:,chr12:113670655-113671680,divergent,FALSE
chr5,54048993,54048994,chr5,54048695,54049421,chr5:54048994:,chr5:54048694-54049421,divergent,TRUE
chr6,7439631,7439632,chr6,7439319,7440045,chr6:7439632:,chr6:7439318-7440045,coacc,FALSE
chr6,90976767,90976768,chr6,90976485,90977421,chr6:90976768:,chr6:90976484-90977421,gwas,TRUE
chr11,118743285,118743286,chr11,118743055,118743481,chr11:118743286:,chr11:118743054-118743481,gwas,TRUE


In [109]:
cat(ex1$V1)

chr5:157506340: chr12:47644392: chr6:1286450: chr9:108966249:

In [110]:
ex1$cell

[1] "naive_b"     "naive_cd4_t" "cMono"       "cyto_nk"